In [3]:
!pip install -q fastapi uvicorn nest_asyncio sentence-transformers


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer, util

# Patch the running event loop
nest_asyncio.apply()

app = FastAPI()
model = SentenceTransformer("nathankim/fine_tuned_resume_matcher")

class TextPair(BaseModel):
    resume: str
    job: str

@app.post("/similarity")
def get_similarity(data: TextPair):
    emb = model.encode([data.resume, data.job], convert_to_tensor=True)
    score = util.pytorch_cos_sim(emb[0], emb[1]).item()
    return {"similarity_score": score}

# For running inside notebooks or VS Code interactive
uvicorn.run(app, host="127.0.0.1", port=8000)
